## Setup

In [4]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch

In [5]:
import torch
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

## GEtting data

In [6]:
import os

# Check if running on Kaggle
if not os.path.exists('datasets/data.csv'):
    # If datasets folder doesn't exist, assume running on Kaggle
    !git clone REPOURL 
    !mkdir datasets
    !mv REPONAME/data.csv REPONAME/test.csv datasets/
    !rm -rf INNOV8-2.0
else:
    print("Dataset already exists locally")


Cloning into 'INNOV8-2.0'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 30 (delta 10), reused 1 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (30/30), 425.37 KiB | 6.75 MiB/s, done.


## Fasttext

In [8]:
!ls

datasets


In [9]:
import os

file_name = "cc.en.300.bin.gz"
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/" + file_name

# Check if the file exists
if not os.path.isfile(file_name):
    #     os.system(f"wget {url}")
    print("Not found")
    !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

else:
    print(f"{file_name} already exists, skipping download.")


Not found
--2024-10-11 11:55:01--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.108, 3.163.189.14, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: 'cc.en.300.bin.gz'

cc.en.300.bin.gz    100%[===================>]   4.19G  48.8MB/s    in 97s     

2024-10-11 11:56:38 (44.2 MB/s) - 'cc.en.300.bin.gz' saved [4503593528/4503593528]



In [10]:
!rm -rf fasttext_pretrained.bin

In [11]:
# !gunzip cc.en.300.bin.gz
# print("Extracted")


import os

file_name = "cc.en.300.bin"
# url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/" + file_name

# Check if the file exists
if not os.path.isfile(file_name):
    #     os.system(f"wget {url}")
    print("Not found")
    !gunzip cc.en.300.bin.gz

else:
    print(f"{file_name} already exists, skipping download.")


Not found


In [12]:
!ls

cc.en.300.bin  datasets


In [13]:
# !rm -rf cc.en.300.bin.gz

In [14]:
# import requests
# import os

# # Define the URL for the FastText model
# url = 'https://fasttext.cc/docs/en/crawl-vectors.html'  # Replace with the direct link to the model
# model_filename = 'fasttext_pretrained.bin'  # Desired name for the model file

# # Create a function to download the model
# def download_fasttext_model(url, filename):
#     # Send a GET request to the URL
#     response = requests.get(url, stream=True)

#     # Check if the request was successful
#     if response.status_code == 200:
#         with open(filename, 'wb') as f:
#             for chunk in response.iter_content(chunk_size=1024):
#                 f.write(chunk)
#         print(f"Model downloaded successfully as '{filename}'")
#     else:
#         print(f"Failed to download model: {response.status_code}")

# # Call the function to download the FastText model
# download_fasttext_model('https://fasttext.cc/docs/en/crawl-vectors.html', model_filename)


In [15]:
model_filename = 'cc.en.300.bin'

In [16]:
import fasttext

# Load the pre-trained FastText model
ft_model = fasttext.load_model(model_filename)  # Ensure the file is in the current directory
print("FastText model loaded successfully.")


FastText model loaded successfully.


In [17]:
!ls

cc.en.300.bin  datasets


In [18]:
# from transformers import BertTokenizer, BertModel
# import torch

# # Define device
# device = 'cpu'

# # Load the BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)  # Move model to device


In [19]:
raw_train_data = pd.read_csv("datasets/data.csv")

In [20]:
train_data, test_data = train_test_split(raw_train_data, test_size=0.2, random_state=42, stratify=raw_train_data['species'])


In [21]:
# df = raw_train_data.copy()

In [22]:
# df = df.iloc[:2,:]

In [23]:
# df

In [24]:
# df

In [25]:
from tqdm import tqdm

In [26]:
'''def get_cls_embedding(texts):
    # Tokenize the input
    inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to device
    
    # Get BERT embeddings
    with torch.no_grad():
        outputs = bert_model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # Extract the [CLS] token embedding
        return cls_embedding  # shape is 1x768'''

def get_fasttext_embedding(texts):
    # Get embeddings for each text using FastText
    embeddings = []
    
    for text in texts:
        # Get word vectors for the text
        words = text.split()  # Simple split, consider using more sophisticated tokenization
        word_vectors = [ft_model.get_word_vector(word) for word in words]
        
        # Compute the mean of word vectors to represent the entire text
        if word_vectors:  # Avoid empty input
            mean_vector = np.mean(word_vectors, axis=0)  # Shape will be (300,)
            embeddings.append(mean_vector)
        else:
            embeddings.append(np.zeros(ft_model.get_dimension()) )  # Handle empty text case
            
    embeddings = np.array(embeddings)
    # Convert the list of embeddings to a PyTorch tensor
    return torch.tensor(embeddings)  # Shape will be (num_texts, 300)


In [34]:



def get_all_embeddings(texts):
    # Define batch size
    batch_size = 32  # Adjust based on your memory capacity

    # Initialize list to store all embeddings
    all_embeddings = []

    # Process texts in batches
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i + batch_size]
        #         batch_embeddings = get_cls_embedding(batch_texts)
        batch_embeddings = get_fasttext_embedding(batch_texts)
        all_embeddings.append(batch_embeddings)

    # Concatenate all embeddings into a single tensor
    embeddings_tensor = torch.cat(all_embeddings, dim=0)
    return embeddings_tensor

In [36]:
# 

from tqdm import tqdm

In [29]:
# from tqdm.notebook import tqdm
# import time

# # Example: Simulating a task with tqdm in a notebook
# for i in tqdm(range(10)):
#     time.sleep(0.5)  # Simulate some work with time.sleep
#     print("hi")


In [30]:
# import time
# from IPython.display import display
# import ipywidgets as widgets

# # Create a progress bar
# progress_bar = widgets.IntProgress(min=0, max=10)
# iteration_label = widgets.Label(value="Iteration: 0 / 10")
# percentage_label = widgets.Label(value=" | 0%")

# # Create an HBox to align the progress bar and labels side by side
# hbox = widgets.HBox([progress_bar, widgets.HBox([iteration_label, percentage_label])])

# # Display the HBox
# display(hbox)

# # Simulate a task with the progress bar
# for i in range(10):
#     time.sleep(0.5)  # Simulate work
#     progress_bar.value = i + 1  # Update progress
#     iteration_label.value = f"Processing: {i + 1} / 10"  # Update iteration
#     percentage_label.value = f"  |  {  (i + 1) * 10}%"  # Update percentage

# # Optionally, indicate completion
# iteration_label.value = "Iteration: 10 / 10"
# percentage_label.value = "100%"


In [31]:
# import time
# from IPython.display import display, clear_output
# import ipywidgets as widgets

# def create_progress_bar(total_iterations):
#     # Create a progress bar
#     progress_bar = widgets.IntProgress(min=0, max=total_iterations)
#     iteration_label = widgets.Label(value=f"Iteration: 0 / {total_iterations}")
#     percentage_label = widgets.Label(value="0%")

#     # Create an HBox to align the progress bar and labels side by side
#     hbox = widgets.HBox([progress_bar, widgets.VBox([iteration_label, percentage_label])])
#     display(hbox)

#     return progress_bar, iteration_label, percentage_label

# def update_progress(progress_bar, iteration_label, percentage_label, current_iteration):
#     progress_bar.value = current_iteration + 1  # Update progress
#     iteration_label.value = f"Iteration: {current_iteration + 1} / {progress_bar.max}"  # Update iteration
#     percentage_label.value = f"{(current_iteration + 1) * 100 // progress_bar.max}%"  # Update percentage

# # Example usage
# def example_task(total_iterations):
#     progress_bar, iteration_label, percentage_label = create_progress_bar(total_iterations)

#     for i in range(total_iterations):
#         time.sleep(0.5)  # Simulate work
#         update_progress(progress_bar, iteration_label, percentage_label, i)

#     # Optionally, indicate completion
#     iteration_label.value = f"Iteration: {total_iterations} / {total_iterations}"
#     percentage_label.value = "100%"

# # Call the function for a task
# example_task(10)


In [37]:

texts = train_data['message'].tolist()
train_embed = get_all_embeddings(texts)


100%|██████████| 13/13 [00:00<00:00, 121.40it/s]


In [38]:

texts = test_data['message'].tolist()
test_embed = get_all_embeddings(texts)


100%|██████████| 4/4 [00:00<00:00, 230.12it/s]


In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
train_embed = train_embed.to(device)
test_embed = test_embed.to(device)

In [41]:
# df

In [42]:
# df = train_data

In [43]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [44]:
# Define lists of feature names
numerical_feature_names = ['fingers',]
categorical_feature_names = ['tail',]
output_feature_name = 'species'

In [45]:
1

1

In [46]:
import torch
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Initialize encoders
one_hot_encoder = OneHotEncoder(sparse=False, drop=None)
label_encoder = LabelEncoder()

def process_dataframe(df, fit_encoders=True):
    # Extract features
    numerical_features = df[numerical_feature_names].values
    categorical_features = df[categorical_feature_names].values
    output_labels = df[output_feature_name].values

    # Convert numerical features to a tensor and move to device
    numerical_tensor = torch.tensor(numerical_features, dtype=torch.float32).to(device)

    # One-hot encode categorical features
    if fit_encoders:
        one_hot_encoded = one_hot_encoder.fit_transform(categorical_features)
    else:
        one_hot_encoded = one_hot_encoder.transform(categorical_features)
    categorical_tensor = torch.tensor(one_hot_encoded, dtype=torch.float32).to(device)

    # Ordinal encode the output feature
    if fit_encoders:
        output_encoded = label_encoder.fit_transform(output_labels)
    else:
        output_encoded = label_encoder.transform(output_labels)
    output_tensor = torch.tensor(output_encoded, dtype=torch.long).to(device)  # Use long type for class indices

    # Create mapping dictionary for output labels
    if fit_encoders:
        label_to_index = {label: idx for idx, label in enumerate(label_encoder.classes_)}
        index_to_label = {idx: label for idx, label in enumerate(label_encoder.classes_)}
    else:
        label_to_index = None
        index_to_label = None

    # Concatenate tensors (assuming embeddings_tensor is available)
    if fit_encoders:
        combined_features = torch.cat((train_embed, numerical_tensor, categorical_tensor), dim=1)
    else:
        combined_features = torch.cat((test_embed, numerical_tensor, categorical_tensor), dim=1)

    inputs = combined_features
    labels = output_tensor
    
    return inputs, labels, label_to_index, index_to_label

# Process training data
train_inputs, train_labels, train_label_to_index, train_index_to_label = process_dataframe(train_data, fit_encoders=True)

# Process test data
test_inputs, test_labels, _, _ = process_dataframe(test_data, fit_encoders=False)

# Now you have processed inputs and labels for both training and test datasets
print("Training inputs shape:", train_inputs.shape)
print("Training labels shape:", train_labels.shape)
print("Test inputs shape:", test_inputs.shape)
print("Test labels shape:", test_labels.shape)


Training inputs shape: torch.Size([400, 303])
Training labels shape: torch.Size([400])
Test inputs shape: torch.Size([100, 303])
Test labels shape: torch.Size([100])


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [47]:
train_data

,message,fingers,tail,species
484,lazeron robonix nanozom,4,no,Cybex
183,dredzx ragex fearix,0,no,Emotivor
116,celestar novum warpz,4,no,Quixnar
120,nebuz kometa lunox novax astron pulsar meteorn...,5,yes,Quixnar
437,astron novax novax pulsar warpz norax cosmix v...,3,no,Quixnar
...,...,...,...,...
86,novarr stardux celestar corp quasar nebulax nx...,3,yes,Quixnar
396,codex cywrex lazeron cybron mechan mechan nano...,0,no,Cybex
328,datax codex lazeron digitron cybrpx,1,yes,Cybex
108,awezom ekstax angstix euphorix excitar shamex ...,3,no,Sentire


In [48]:
# train_index_to_label

In [49]:
# # Concatenate tensors
# combined_features 


In [50]:
# combined_features

In [51]:
# inputs.shape

In [52]:

# num_classes = len(label_to_index)

In [53]:
# num_classes

In [54]:
# Raise ZeroDivisionError

In [55]:
import torch.nn as nn

# Define a simple MLP for classification
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define the model
input_size = train_inputs.shape[1]  # Adjusted to match your data
hidden_size = 256
num_classes = len(train_index_to_label)  # Number of output classes

model = MLPClassifier(input_size, hidden_size, num_classes).to(device)


In [56]:
summary(model=model, 
        )

Layer (type:depth-idx)                   Param #
MLPClassifier                            --
├─Linear: 1-1                            77,824
├─ReLU: 1-2                              --
├─Linear: 1-3                            2,570
Total params: 80,394
Trainable params: 80,394
Non-trainable params: 0

In [57]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions."""
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_true)) * 100
    return acc

loss_fn = nn.CrossEntropyLoss()

In [58]:
WRITE_LOG_FLAG = None

In [59]:
WRITE_LOG_FLAG

In [60]:
import torch as t
from tqdm.auto import tqdm
from timeit import default_timer as timer

def test_step(test_inputs, test_labels, m0, loss_fn, accuracy_fn, batch_size, device):
    m0.eval()
    test_loss, test_acc = 0, 0
    num_batches = (len(test_inputs) + batch_size - 1) // batch_size  # Calculate number of batches

    with t.inference_mode():
        for start in range(0, len(test_inputs), batch_size):
            end = min(start + batch_size, len(test_inputs))
            X_batch = test_inputs[start:end].to(device)
            y_batch = test_labels[start:end].to(device)

            test_pred = m0(X_batch)
            test_loss += loss_fn(test_pred, y_batch).item()
            test_acc += accuracy_fn(y_batch, test_pred.argmax(dim=1))

    test_loss /= num_batches
    test_acc /= num_batches
    return test_loss, test_acc

def train_step(train_inputs, train_labels, test_inputs, test_labels, m0, optimizer, loss_fn, accuracy_fn, batch_size, validate_flag,WRITE_LOG_FLAG, device, epoch, print_flag  = True):
    start_time = timer()
    train_loss, acc = 0, 0
    m0.train()
    num_batches = (len(train_inputs) + batch_size - 1) // batch_size  # Calculate number of batches

    for start in range(0, len(train_inputs), batch_size):
        end = min(start + batch_size, len(train_inputs))
        X_batch = train_inputs[start:end].to(device)
        y_batch = train_labels[start:end].to(device)

        y_pred = m0(X_batch)
        loss = loss_fn(y_pred, y_batch)
        train_loss += loss.item()
        acc += accuracy_fn(y_batch, y_pred.argmax(dim=1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= num_batches
    acc /= num_batches

    if validate_flag:
        test_loss, test_acc = test_step(test_inputs, test_labels, m0, loss_fn, accuracy_fn, batch_size, device)
        end_time = timer()
        content = (f"Epoch: {epoch}  | Train Loss: {train_loss:.3f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.3f}, Acc: {test_acc:.2f}% | Time taken: {(end_time - start_time):.3f} sec")
        
        if print_flag:
            print(content)
        if WRITE_LOG_FLAG : 
            # Open the file in write mode ('w')
            if epoch == 1 : 
                filemode = 'w+'
            else:
                filemode = 'a+' 
            with open(f'model/{LOG_FILE_NAME}.txt', filemode) as file:
                file.write(content+'\n')

    else:
        test_loss, test_acc = -1, -1

    

In [61]:
# Define constants
BATCH_SIZE = 32
NUM_EPOCHS = 100
EVAL_INTERVAL = 10
LR  = 0.2
LOG_FILE_NAME = 'trainlog'
WRITE_LOG_FLAG = True

In [62]:
!mkdir model

In [68]:
import torch as t
from tqdm.auto import tqdm



def reset(device, hidden_size):
    """Reset the model and optimizer."""
    t.manual_seed(42)
    
    model = MLPClassifier(input_size, hidden_size, num_classes).to(device)
    optimizer = t.optim.SGD(params=model.parameters(), lr=LR)
    return model, optimizer


def train_model(model, optimizer,num_epochs,  print_flag = True,validate_flag = None, batch_size = BATCH_SIZE):
    input_validate_flag = validate_flag
    
    device = t.device('cuda' if t.cuda.is_available() else 'cpu')

    #     # Initialize model and optimizer
    #     model, optimizer = reset(device, hidden_size)
   
    for epoch in tqdm(range(1, num_epochs + 1)):
        
        # Train the model
        if input_validate_flag is not None:
            validate_flag = input_validate_flag
        elif epoch == 1 or epoch == NUM_EPOCHS or epoch%EVAL_INTERVAL == 0:
            validate_flag = True
        else:
            validate_flag = False
        #         print(validate_flag)
        train_step(
            train_inputs, train_labels, 
            test_inputs, test_labels, 
            model, optimizer, loss_fn, 
            accuracy_fn, batch_size, 
            validate_flag=validate_flag, 
            WRITE_LOG_FLAG = WRITE_LOG_FLAG,
            device=device, 
            epoch=epoch,
            print_flag = print_flag
        )
        
    return model , optimizer


In [64]:
hidden_size

256

In [65]:
# kyu

In [66]:
2

2

In [69]:
# Define constants
BATCH_SIZE = 2048*2
NUM_EPOCHS = 5000
EVAL_INTERVAL = NUM_EPOCHS // 10
LR  = 0.05
hidden_size = 128

t.manual_seed(42)

model = MLPClassifier(input_size, hidden_size, num_classes).to(device)
optimizer = t.optim.Adam(params=model.parameters(), lr=LR)


model, optimizer = train_model(model, optimizer, print_flag = True, num_epochs = NUM_EPOCHS)


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch: 1  | Train Loss: 2.309, Acc: 9.75% | Test Loss: 2.201, Acc: 15.00% | Time taken: 0.132 sec
Epoch: 500  | Train Loss: 0.000, Acc: 100.00% | Test Loss: 1.373, Acc: 82.00% | Time taken: 0.003 sec
Epoch: 1000  | Train Loss: 0.000, Acc: 100.00% | Test Loss: 1.559, Acc: 81.00% | Time taken: 0.004 sec
Epoch: 1500  | Train Loss: 0.000, Acc: 100.00% | Test Loss: 1.673, Acc: 81.00% | Time taken: 0.003 sec
Epoch: 2000  | Train Loss: 0.000, Acc: 100.00% | Test Loss: 1.759, Acc: 81.00% | Time taken: 0.003 sec
Epoch: 2500  | Train Loss: 0.000, Acc: 100.00% | Test Loss: 1.830, Acc: 81.00% | Time taken: 0.003 sec


KeyboardInterrupt: 

In [70]:
# test_step(test_inputs, test_labels, model, loss_fn, accuracy_fn, BATCH_SIZE, device)[1]

In [71]:
print("Hypertuning")

Hypertuning


In [72]:
import optuna


def objective(trial):
    # Hyperparameter tuning using Optuna
    BATCH_SIZE = trial.suggest_int("batch_size", 512, 1048, step=512)  # Tune BATCH_SIZE
    NUM_EPOCHS = trial.suggest_int("num_epochs", 1000, 10000, step=1000)  # Tune NUM_EPOCHS
    LR = trial.suggest_float("lr", 1e-5, 1e-1, log = True)  # Tune learning rate
    hidden_size = trial.suggest_int("hidden_size", 64, 512, step=64)  # Tune hidden_size

    # Initialize model and optimizer
    t.manual_seed(42)
    model = MLPClassifier(input_size, hidden_size, num_classes).to(device)
    optimizer = t.optim.Adam(params=model.parameters(), lr=LR)

    # DataLoader with the current BATCH_SIZE
    # train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    # Train model and get accuracy or loss
    #     print("training")
    model, optimizer = train_model(model, optimizer, num_epochs=NUM_EPOCHS, print_flag = False,validate_flag = False, batch_size = BATCH_SIZE)
    #     print("evaluating")
    # Evaluate the model and return a metric to minimize or maximize
    # For example, return validation loss or accuracy
    # return validation_loss  # Replace with your evaluation logic

    # For demonstration purposes, let's assume we return a random value as the loss
    test_loss, test_acc = test_step(test_inputs, test_labels, model, loss_fn, accuracy_fn, BATCH_SIZE, device)
    print(f"Trial {trial}: Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    return test_loss # Replace this with your actual evaluation metric

def tune_hyperparameters():
    study = optuna.create_study(direction="minimize")  # Change to "maximize" if you're maximizing an accuracy metric
    study.optimize(objective, n_trials=3)  # Number of trials for tuning

    print("Best hyperparameters: ", study.best_params)
    print("Best trial value: ", study.best_value)

    # Delete the study before returning
    optuna.delete_study(study_name=study_name)
# Run hyperparameter tuning
# tune_hyperparameters()


In [73]:
1

1

In [76]:
import optuna
import torch as t
from multiprocessing import Pool


def parallel_optimize(num_trials):
    study_name = 'new'
    
    
    study = optuna.create_study(direction="minimize", study_name=study_name)  # Change to "maximize" if you're maximizing an accuracy metric
    
    # Run trials in parallel
    print("Hypertuning")
    study.optimize(objective, n_trials=num_trials, n_jobs=-1 )  # n_jobs=-1 means use all available cores
    
    print("Best hyperparameters: ", study.best_params)
    
    print("Best trial value: ", study.best_value)
    
    # Delete the study before returning
    #     optuna.delete_study(study_name=study_name)
    return  study.best_params

# Run hyperparameter tuning in parallel
best_params = parallel_optimize(num_trials=50)  # Adjust the number of trials as needed
best_params


[I 2024-10-11 12:02:24,467] A new study created in memory with name: new


Hypertuning


  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

[I 2024-10-11 12:13:35,995] Trial 48 finished with value: 0.6608646512031555 and parameters: {'batch_size': 512, 'num_epochs': 4000, 'lr': 2.0728427139691583e-05, 'hidden_size': 384}. Best is trial 37 with value: 0.3870869576931.


Trial <optuna.trial._trial.Trial object at 0x7d03148977c0>: Test Loss: 0.6609, Test Accuracy: 87.0000


[I 2024-10-11 12:13:37,785] Trial 49 finished with value: 0.6753148436546326 and parameters: {'batch_size': 512, 'num_epochs': 4000, 'lr': 2.0344141166762413e-05, 'hidden_size': 384}. Best is trial 37 with value: 0.3870869576931.


Trial <optuna.trial._trial.Trial object at 0x7d0314895bd0>: Test Loss: 0.6753, Test Accuracy: 87.0000
Best hyperparameters:  {'batch_size': 512, 'num_epochs': 4000, 'lr': 5.244142622890294e-05, 'hidden_size': 320}
Best trial value:  0.3870869576931


{'batch_size': 512,
 'num_epochs': 4000,
 'lr': 5.244142622890294e-05,
 'hidden_size': 320}

In [77]:
best_params

{'batch_size': 512,
 'num_epochs': 4000,
 'lr': 5.244142622890294e-05,
 'hidden_size': 320}

In [78]:
from types import SimpleNamespace
# Convert the dictionary to a SimpleNamespace object
best_params = SimpleNamespace(**best_params)
best_params

namespace(batch_size=512,
          num_epochs=4000,
          lr=5.244142622890294e-05,
          hidden_size=320)

In [79]:
EVAL_INTERVAL

500

In [91]:
BATCH_SIZE = best_params.batch_size
# NUM_EPOCHS = best_params.num_epochs
NUM_EPOCHS = 2400
LR = best_params.lr
hidden_size = best_params.hidden_size
EVAL_INTERVAL = NUM_EPOCHS // 10
# EVAL_INTERVAL = 20

t.manual_seed(42)
model = MLPClassifier(input_size, hidden_size, num_classes).to(device)
optimizer = t.optim.Adam(params=model.parameters(), lr=LR)

# DataLoader with the current BATCH_SIZE
# train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Train model and get accuracy or loss
# print("training")
model, optimizer = train_model(model, optimizer, num_epochs=NUM_EPOCHS, print_flag = True,validate_flag = None, batch_size = BATCH_SIZE)
# print("evaluating")
# Evaluate the model and return a metric to minimize or maximize
# For example, return validation loss or accuracy
# return validation_loss  # Replace with your evaluation logic

# For demonstration purposes, let's assume we return a random value as the loss
# test_loss, test_acc = test_step(test_inputs, test_labels, model, loss_fn, accuracy_fn, BATCH_SIZE, device)
# print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
# return test_loss # Replace this with your actual evaluation metric


  0%|          | 0/2400 [00:00<?, ?it/s]

Epoch: 1  | Train Loss: 2.295, Acc: 10.25% | Test Loss: 2.298, Acc: 10.00% | Time taken: 0.007 sec
Epoch: 240  | Train Loss: 2.099, Acc: 46.00% | Test Loss: 2.112, Acc: 46.00% | Time taken: 0.004 sec
Epoch: 480  | Train Loss: 1.792, Acc: 63.75% | Test Loss: 1.824, Acc: 60.00% | Time taken: 0.004 sec
Epoch: 720  | Train Loss: 1.461, Acc: 75.50% | Test Loss: 1.510, Acc: 72.00% | Time taken: 0.006 sec
Epoch: 960  | Train Loss: 1.165, Acc: 82.50% | Test Loss: 1.224, Acc: 82.00% | Time taken: 0.004 sec
Epoch: 1200  | Train Loss: 0.924, Acc: 85.50% | Test Loss: 0.989, Acc: 84.00% | Time taken: 0.004 sec
Epoch: 1440  | Train Loss: 0.740, Acc: 87.25% | Test Loss: 0.811, Acc: 86.00% | Time taken: 0.004 sec
Epoch: 1680  | Train Loss: 0.605, Acc: 88.00% | Test Loss: 0.683, Acc: 87.00% | Time taken: 0.006 sec
Epoch: 1920  | Train Loss: 0.507, Acc: 89.25% | Test Loss: 0.593, Acc: 87.00% | Time taken: 0.004 sec
Epoch: 2160  | Train Loss: 0.436, Acc: 90.00% | Test Loss: 0.531, Acc: 88.00% | Time take

In [81]:
# raise Zero

In [95]:
!ls

cc.en.300.bin  datasets


In [97]:
import torch

In [99]:
# !mkdir model

In [100]:


hyperparameters = {
    "BATCH_SIZE": BATCH_SIZE,
    "NUM_EPOCHS": NUM_EPOCHS,
    "EVAL_INTERVAL": EVAL_INTERVAL,
    "LR": LR
}
# Save the model's state dictionary
torch.save(model.state_dict(), 'model/model_state.pth')
torch.save(model, 'model/model.pth')

# Define the file path
file_path = 'model/hyperparameters.txt'

# Open the file in write mode ('w')
with open(file_path, 'w') as file:
    for key, value in hyperparameters.items():
        file.write(f"{key}: {value}\n")

print(f"Hyperparameters saved to {file_path}")


Hyperparameters saved to model/hyperparameters.txt


In [101]:
# !rm -rf model

In [102]:
import zipfile
import os
from IPython.display import FileLink, display

# Path to the file or directory to zip
file_to_zip = 'model'
zip_file_name = 'model_zip.zip'

# Function to zip a directory
def zip_dir(directory, zip_file):
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, directory))

# Zip the file or directory
if os.path.isdir(file_to_zip):
    zip_dir(file_to_zip, zip_file_name)
else:
    with zipfile.ZipFile(zip_file_name, 'w') as zipf:
        zipf.write(file_to_zip, os.path.basename(file_to_zip))

# Display the download link
download_link = FileLink(zip_file_name)
display(download_link)


/kaggle/working/model_zip.zip

In [103]:
# Raise EofError